# Inverse Solve

We solve the inverse problem of identifying the underlying, spatially-varying thermal conductivity $h(x,y)$ given the surface temperature $T(x,y)$ and a constant heat source $q(x,y)=1$ within a unit square domain $[0,1]\times[0,1]$.

The system is described by the steady-state Poisson heat equation in two dimensions (2D) and subjected to the following mixed boundary conditions:
 - $T=300K$ at the bottom.
 - Insulated walls, i.e. $\frac{\partial T}{\partial \hat{n}}$ ($\hat{n}$ being the surface unit vector), on the other three sides.

To find $h$, we will minimize the following cost function J:

$J=\frac{1}{2}\left[\frac{\left(T-T_{\rm obs}\right)^2}{\sigma^2}+\alpha|\nabla^2 h|\right]$,

using the conjugate-gradient-descend method. Since the forward solver does not provide the gradient, we will solve for the gradient using the adjoint method.

In [ ]:
import os
import h5py as h5
import numpy as np

from mpi4py import MPI
from petsc4py import PETSc

from forward_solver import SteadyHeat2DForwardSolver as ForwardSolver
from adjoint_solver import SteadyHeat2DAdjointSolver as AdjointSolver
from tao_solver import SteadyHeat2DTAOSolver

## Load saved output from disk

In [ ]:
test_dir='../test_data/'
test_outfile_name='blackbox_output'
test_outfile_format='.xdmf'

def list_h5(name, obj):
    """
    HDF5 IO helper
    """
    if isinstance(obj, h5.Dataset):
        print(f"{name}    Dataset, shape={obj.shape}, dtype={obj.dtype}")
    else:
        print(f"{name}    Group")

test_outfile_format='.h5'
with h5.File(os.path.join(test_dir,test_outfile_name+test_outfile_format), 'r') as f:
    f.visititems(list_h5)
    T_noiseless = f['Function/Temperature/0'][()]
    T_noisy = f['Function/ObservedTemperature/0'][()]

### Forward solver

#### Define domain

In [ ]:
nmesh=128
mesh_type='quadrilateral'

#### Define heat source

In [ ]:
q=1.0 # constant heat source

#### Define boundary condition

In [ ]:
T_bottom=300. # in Kelvin, Dirichlet boundary condition on the bottom

#### Define solver settings

In [ ]:
petsc_options = {
     'ksp_type': 'cg',
     'pc_type': 'hypre',
     'ksp_rtol': 1e-12,
}

#### Define intial guess for the "unknown" thermal conductivity

In [ ]:
h_init = 4.*np.ones((nmesh,nmesh), dtype=float)
idx=np.arange(nmesh)
xx,yy=np.meshgrid(idx,idx,indexing='ij')
hmesh_init=np.column_stack([xx.ravel(),yy.ravel(),h_init.ravel()])

#### Define forward solver

In [ ]:
fwd_solver=ForwardSolver(nmesh=nmesh, mesh_type=mesh_type, \
                                          h=hmesh_init, q=q,\
                                          DBC_value=T_bottom,
                                          petsc_opts=petsc_options)
fwd_solver.solve()

#### Define adjoint solver

In [ ]:
## Defaul parameter values for the adjoint solver
sigma = 1e-3 ## noise std.
alpha = 5e-3 ## amplitude of regularization term
adj_solver = AdjointSolver(fwd_solver, T_noisy.squeeze(axis=1), sigma=sigma, alpha=alpha, DBC_value=T_bottom, petsc_opts=petsc_options)
adj_solver.solve()

In [ ]:
def default_monitor(tao):
    it  = tao.getIterationNumber()
    obj = tao.getObjectiveValue()
    print(f"[TAO] iter={it:3d};  J={obj:.6e}.")

solvers = ['bncg','blmvm']
ls_algorithm=['more-thuente','armijo']
gatol=1e-5
grtol=1e-5
gttol=1e-6
verbose=1
tao_solver = SteadyHeat2DTAOSolver(fwd_solver, adj_solver, tao_type=solvers[1], ls_algorithm=ls_algorithm[1],
                                                 use_logh=True, h_min=0.1, h_max=10.,
                                                 gatol=gatol, grtol=grtol, gttol=gttol, mit=1000,
                                                 monitor=default_monitor, verbose=1)

In [ ]:
sol=tao_solver.solve()
print("Solution h(x,y)=",sol)

In [ ]:
from plotting_utils import plot_scalar_mesh
plot_scalar_mesh(fwd_solver.V.mesh,sol,'h_sol(x,y)',cmap='plasma',user_scalar_bar={"fmt": "%.4f"})

In [ ]:
def h_func(x):
    return 1.0 + 6.0*x[0]**2 + x[0]/(1.0 + 2.0*x[1]**2)
blackbox_solver=ForwardSolver(nmesh=nmesh, mesh_type=mesh_type, \
                                          h=h_func, q=q,\
                                          DBC_value=T_bottom,
                                          petsc_opts=petsc_options)

rel_diff=(sol-blackbox_solver.h.function.x.array)/blackbox_solver.h.function.x.array
plot_scalar_mesh(fwd_solver.V.mesh,rel_diff,'(h_sol-h_true)/h_true',cmap='plasma',user_scalar_bar={"fmt": "%.4f"})